<a href="https://colab.research.google.com/github/git933/crawling/blob/main/03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터프레임 생성 후 다양한 방식으로 저장

In [ ]:
import pandas as pd

# 표 (데이터프레임) 만들기
no = []
subject_name = []

no.append(1)
no.append(2)
no.append(3)

subject_name.append('수학')
subject_name.append('과학')
subject_name.append('빅데이터')

subject = pd.DataFrame()
subject['과목번호'] = no
subject['과목명'] = subject_name
print(subject)

# csv 형식 저장
subject.to_csv("C:/Users/tjs70/Desktop/crawling1", encoding="utf-8-sig", index=False)

# xls 형식 저장
subject.to_excel("C:/Users/tjs70/Desktop/crawling1", index=False)

네이버 블로그 크롤링

In [ ]:
# 네이버 블로그 정보 수집하여 저장하기
print("=" * 80)
print(" 연습문제 : 저장할 내용을 목록으로 만들어서 xlsx , csv 형식으로 저장하기")
print("=" * 80)

# 라이브러리 로딩, 검색어 입력
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import pandas as pd

query_txt = input('1. 크롤링할 키워드는 무엇입니까? : ')
cnt = int(input('2. 수집할 데이터는 몇 건입니까? : '))
page_cnt = math.ceil(cnt / 30)  # 30건의 데이터가 한 페이지에 있음

fc_name = input('3. csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요 (예 : c:/temp/naver.csv) : ')
fx_name = input('4. xlsx 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요 (예 : c:/temp/naver.xlsx) : ')

# 웹 브라우저 실행
path = "C:/Temp/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('http://www.naver.com')
time.sleep(2)

# 검색 후 블로그 선택
element = driver.find_element_by_id("query")
element.send_keys(query_txt)
element.submit()

driver.find_element_by_link_text("VIEW").click()
time.sleep(2)

driver.find_element_by_link_text("블로그").click()

# 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장
no2 = []  # 게시글 번호
title2 = []  # 게시물 제목
contents2 = []  # 게시글 내용
bdate2 = []  # 작성 일자
nick2 = []  # 닉네임

# 목록에 있는 내용을 파일에 저장
no = 1

# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(8)

i = 1
while (i <= page_cnt):
    scroll_down(driver)
    i += 1
    print('%s 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요.' % i)

print("\n")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

view_list = soup.find('ul', 'lst_total').find_all('li')

for i in view_list:
    no2.append(no)  # 게시물 번호 리스트에 추가
    print('1. 번호 : ', no)

    all_title = i.find_all('a')
    title = all_title[2].get_text()  # 게시물 제목
    title2.append(title)  # 게시물 제목 리스트에 추가
    print('2. 제목 : ', title)

    try:
        contents = i.find('div', 'api_txt_lines dsc_txt').get_text()  # 게시물 내용
    except:
        contents = '본문 요약 내용이 없습니다'
        contents2.append(contents)  # 게시물 내용 리스트에 추가
        print('3. 요약 내용 : ', contents)
    else:
        contents2.append(contents)  # 게시물 내용 리스트에 추가
        print('3. 요약 내용 : ', contents)

    bdate = i.find('span', 'sub_time sub_txt').get_text()  # 작성 일자
    bdate2.append(bdate)  # 작성 일자 리스트에 추가
    print('4. 작성 일자 : ', bdate)

    print("\n")

    if no == cnt:
        break

    no += 1

# 출력 결과를 표 형태로 만들기
naver_blog = pd.DataFrame()
naver_blog['번호'] = no2
naver_blog['제목'] = title2
naver_blog['요약내용'] = contents2
naver_blog['작성일자'] = bdate2

# csv 형태로 저장
naver_blog.to_csv(fc_name, encoding="utf-8-sig", index=False)
print(" csv 파일 저장 경로 : %s" % fc_name)

# 엑셀 형태로 저장
naver_blog.to_excel(fx_name, index=False)
print(" xls 파일 저장 경로: %s" % fx_name)